In [1]:
from deepgram import Deepgram

# Your Deepgram API Key
DEEPGRAM_API_KEY = ''


In [2]:
# Define your desired data structure.
from langchain_community.llms import Ollama
from typing import List

from langchain.output_parsers import PydanticOutputParser
from langchain.prompts import PromptTemplate
from langchain_core.pydantic_v1 import BaseModel, Field, validator
from langchain_openai import ChatOpenAI
from langchain_core.messages import HumanMessage
from langchain_mistralai.chat_models import ChatMistralAI
from langchain.schema.output_parser import StrOutputParser

    
class LLM:
    def __init__(self, local=False):
        
        if local:
            llm = Ollama(model="mistral:instruct", temperature=0.6)
        else:
            llm = ChatMistralAI(mistral_api_key='', model = 'mistral-tiny')
        

        prompt = PromptTemplate(
            template="Answer the user query.\n{query}\n",
            input_variables=["query"],
            
        )
        


        chain = prompt | llm |StrOutputParser()
        self.chain = chain
    
    def get(self, text):
        ans = self.chain.invoke({"query": f''' {text}'''})
                
        return ans
            


In [3]:
lm = LLM(local=False)
#lm.get('what is the capital of india')

In [4]:
lm.get('what is the capital of india')

'The capital city of India is New Delhi. It is a city that holds great historical and cultural significance and is home to numerous landmarks and institutions. New Delhi replaced Calcutta as the capital of India in 1911 when British India was ruled by the British Empire. Since India gained its independence from British rule in 1947, New Delhi has continued to serve as the capital city.'

In [5]:
import os
os.environ['DEEPGRAM_API_KEY'] = DEEPGRAM_API_KEY

In [6]:
import requests
import io
from pydub import AudioSegment
from pydub.playback import play

def stream_audio(text):

    DEEPGRAM_URL = "https://api.deepgram.com/v1/speak?model=aura-asteria-en"
    headers = {
        "Authorization": f"Token {DEEPGRAM_API_KEY}",
        "Content-Type": "application/json"
    }
    payload = {
        "text": text
    }

    # Create an in-memory byte buffer to store the audio stream
    audio_buffer = io.BytesIO()

    # Make a POST request to Deepgram API and stream the response
    with requests.post(DEEPGRAM_URL, stream=True, headers=headers, json=payload) as r:
        for chunk in r.iter_content(chunk_size=1024):
            if chunk:
                # Write the chunk to the byte buffer
                audio_buffer.write(chunk)

    # Reset the buffer position to the beginning
    audio_buffer.seek(0)

    # Load the byte buffer as an AudioSegment
    audio = AudioSegment.from_mp3(audio_buffer)

    # Play the audio
    play(audio)

    print("Audio playback started.")


In [7]:
import pyaudio
import wave


In [8]:
# main.py (python example)

import os
from dotenv import load_dotenv

from deepgram import (
    DeepgramClient,
    PrerecordedOptions,
)

load_dotenv()
DEEPGRAM_API_KEY = ''
# URL to the audio file
# main.py (python example)

import os
from dotenv import load_dotenv

from deepgram import (
    DeepgramClient,
    PrerecordedOptions,
    FileSource,
)

load_dotenv()

# Path to the audio file
AUDIO_FILE = "output.wav"

API_KEY = DEEPGRAM_API_KEY


def transcribe():
    try:
        # STEP 1 Create a Deepgram client using the API key
        deepgram = DeepgramClient(API_KEY)

        with open(AUDIO_FILE, "rb") as file:
            buffer_data = file.read()

        payload: FileSource = {
            "buffer": buffer_data,
        }

        #STEP 2: Configure Deepgram options for audio analysis
        options = PrerecordedOptions(
            model="nova-2",
            smart_format=True,
        )

        # STEP 3: Call the transcribe_file method with the text payload and options
        response = deepgram.listen.prerecorded.v("1").transcribe_file(payload, options)

        # STEP 4: Print the response
        question = response.results.channels[0].alternatives[0].transcript
        if question is not None and question != '':
            print(question)

            res = lm.get(response.results.channels[0].alternatives[0].transcript)
            print(res)
            stream_audio(res)
        

    except Exception as e:
        print(f"Exception: {e}")






In [9]:
# while True:
#     record_audio()
#     transcribe()

In [10]:
# >>> import pyaudio
# >>> audio = pyaudio.PyAudio()
# >>> audio.get_device_count()
# 3
# >>> audio.get_device_info_by_index(2)
# {..., 'name': 'Loopback Audio', ...}
# # use input_device_index=2


In [12]:
# main.py (python example)
import pyaudio
import wave
import os
from dotenv import load_dotenv

import speech_recognition as sr
import pyttsx3
import pywhatkit
import datetime
import wikipedia
import pyjokes




def listen():
    WAVE_OUTPUT_FILENAME = "output.wav"
    FORMAT = pyaudio.paInt16
    CHANNELS = 1
    RATE = 16000
    CHUNK = 1024
    RECORD_SECONDS = 3
    try:
        r=sr.Recognizer()
        
        with sr.Microphone(device_index=1) as source:
            r.adjust_for_ambient_noise(source)
            # r.energy_threshold()
            print("say anything : ")
            audio= r.listen(source)
            waveFile = wave.open(WAVE_OUTPUT_FILENAME, 'wb')
            waveFile.setnchannels(CHANNELS)
            waveFile.setsampwidth(audio.sample_width)
            waveFile.setframerate(audio.sample_rate)
            waveFile.writeframes(audio.frame_data)
            waveFile.close()
    except Exception as e:
        print(e)
    return None

In [17]:
print(sr.Microphone.list_microphone_names())
while True:
    listen()
    transcribe()

['BenQ GW2780', 'MacBook Pro Microphone', 'MacBook Pro Speakers', 'ZoomAudioDevice']
say anything : 
What is the meaning of life?
The meaning of life is a question that has puzzled philosophers, theologians, and thinkers throughout history. There is no definitive answer, as different people, cultures, and religions have various beliefs and interpretations. Some believe the meaning of life is to seek happiness, knowledge, love, or personal growth. Others believe it is to serve a greater purpose, such as a god or the universe. Ultimately, the meaning of life may be something that each individual must discover for themselves.
Audio playback started.
say anything : 
What is the capital of India?
The capital city of India is New Delhi. It is important to note that New Delhi is not a state in India, but rather the capital territory where the central government administers the country. It is located in the northern part of India and is home to numerous historical sites, important institutions

KeyboardInterrupt: 